In [1]:
import os, sys
import pandas as pd
from pandasql import sqldf

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

import scripts.read_data_from_db as rd
import scripts.utils as util

In [2]:
df = rd.read_data(table_name='processed_data')

INFO:scripts.read_data_from_db:Data fetched succesfully


In [3]:
df.shape

(150000, 53)

In [4]:
pysqldf = lambda q: sqldf(q, globals())

In [5]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B',
       'Social Me

### Top 10 handsets used by the customers

In [6]:
query = ''' 
    SELECT DISTINCT 
        "Handset Type", 
        COUNT(*) as UsageCount
    FROM df
    GROUP BY "Handset Type" 
    ORDER BY UsageCount DESC
    limit 10
'''

result_df = pysqldf(query)
result_df

,Handset Type,UsageCount
0,Huawei B528S-23A,20323
1,Apple iPhone 6S (A1688),9419
2,Apple iPhone 6 (A1586),9023
3,undefined,8987
4,Apple iPhone 7 (A1778),6326
5,Apple iPhone Se (A1723),5187
6,Apple iPhone 8 (A1905),4993
7,Apple iPhone Xr (A2105),4568
8,Samsung Galaxy S8 (Sm-G950F),4520
9,Apple iPhone X (A1901),3813


### Top 3 handset Manufacturers


In [7]:
query = ''' 
    SELECT DISTINCT 
        "Handset Manufacturer", 
        COUNT(*) as "Number of Users"
    FROM df
    GROUP BY "Handset Manufacturer" 
    ORDER BY "Number of Users" DESC
    limit 3
'''

result_df = pysqldf(query)
result_df

,Handset Manufacturer,Number of Users
0,Apple,60136
1,Samsung,40839
2,Huawei,34423


### Average session Duration by Manufacturer

In [8]:
query = ''' 
    SELECT "Handset Manufacturer", AVG("Dur. (ms)") AS "Avg Session Duration (ms)"
    FROM df
    WHERE "Handset Manufacturer" IN ('Apple', 'Samsung', 'Huawei')
    GROUP BY "Handset Manufacturer";

'''
result_df = pysqldf(query)
result_df

,Handset Manufacturer,Avg Session Duration (ms)
0,Apple,98449.876264
1,Huawei,96095.559568
2,Samsung,121664.988467


### Data volume Analysis by Manufacturer


In [9]:
query = ''' 
   SELECT
    "Handset Manufacturer",
    SUM("Total UL (Bytes)" + "Total DL (Bytes)") AS "Total Data Volume (Bytes)"
    FROM df
    WHERE "Handset Manufacturer" IN ('Apple', 'Samsung', 'Huawei')
    GROUP BY "Handset Manufacturer";
'''
result_df = sqldf(query)
result_df

,Handset Manufacturer,Total Data Volume (Bytes)
0,Apple,2.978488e+13
1,Huawei,1.709393e+13
2,Samsung,2.023676e+13


### Top 3 Handsets per top 3 handset Manufacurers

In [10]:
query = ''' 
    WITH RankedHandsets AS (
    SELECT
        "Handset Manufacturer",
        "Handset Type",
        RANK() OVER (PARTITION BY "Handset Manufacturer" ORDER BY COUNT(*) DESC) AS "Rank"
    FROM df
    WHERE "Handset Manufacturer" IN ('Apple', 'Samsung', 'Huawei')
    GROUP BY "Handset Manufacturer", "Handset Type"
    )
    SELECT
        "Handset Manufacturer",
        "Handset Type",
        "Rank"
    FROM RankedHandsets
    WHERE "Rank" <= 5;
'''

result_df = sqldf(query)
result_df

,Handset Manufacturer,Handset Type,Rank
0,Apple,Apple iPhone 6S (A1688),1
1,Apple,Apple iPhone 6 (A1586),2
2,Apple,Apple iPhone 7 (A1778),3
3,Apple,Apple iPhone Se (A1723),4
4,Apple,Apple iPhone 8 (A1905),5
5,Huawei,Huawei B528S-23A,1
6,Huawei,Huawei E5180,2
7,Huawei,Huawei P20 Lite Huawei Nova 3E,3
8,Huawei,Huawei P20,4
9,Huawei,Huawei Y6 2018,5


### Number of xDR sessions per user

In [11]:
query = ''' 
    SELECT "MSISDN/Number" AS UserIdentifer,
    COUNT(*) AS NumberOfXDRSessions
    FROM df
    GROUP BY "MSISDN/Number"
    ORDER BY NumberOfXDRSessions DESc;
'''

result_df = pysqldf(query)
result_df

,UserIdentifer,NumberOfXDRSessions
0,3.362632e+10,1083
1,3.362578e+10,17
2,3.361489e+10,17
3,3.365973e+10,16
4,3.376054e+10,15
...,...,...
106851,3.360101e+10,1
106852,3.360101e+10,1
106853,3.360100e+10,1
106854,3.360100e+10,1


### number of users who have >= 10 xDR sessions

In [12]:
query = '''
    SELECT COUNT(*) AS num_users
    FROM (
        SELECT COUNT(*) AS NumberOfXDRSessions
        FROM df
        GROUP BY "MSISDN/Number"
        HAVING COUNT(*) >= 10
    ) AS subquery
'''

result_df = pysqldf(query)
result_df

,num_users
0,29


### Top users by session duration

In [13]:
query = ''' 
    SELECT
        "MSISDN/Number" AS UserIdentifier,
        SUM("Dur. (ms)") / 1000 AS TotalSessionDurationInSeconds
    FROM df
    GROUP BY "MSISDN/Number"
    ORDER BY TotalSessionDurationInSeconds DESC;

'''

result_df = pysqldf(query)
result_df

,UserIdentifier,TotalSessionDurationInSeconds
0,3.362632e+10,81238.323
1,3.362578e+10,18553.754
2,3.361489e+10,9966.898
3,3.376054e+10,9279.434
4,3.366716e+10,8744.914
...,...,...
106851,3.365043e+10,7.267
106852,3.369839e+10,7.258
106853,3.365933e+10,7.189
106854,3.366566e+10,7.146


### Peak usage Hours

In [14]:
query = ''' 
    SELECT
        CAST(SUBSTRING(CAST("Start" AS TEXT), INSTR(CAST("Start" AS TEXT), ' ') + 1, INSTR(CAST("Start" AS TEXT), ':') - INSTR(CAST("Start" AS TEXT), ' ') - 1) AS INTEGER) AS HourOfDay,
        COUNT(*) AS NumberOfSessions
    FROM df
    WHERE "Start" IS NOT NULL
    GROUP BY HourOfDay
    ORDER BY NumberOfSessions DESC;

'''

result_df = pysqldf(query) 
result_df

,HourOfDay,NumberOfSessions
0,7,14176
1,8,9065
2,6,8513
3,2,8296
4,4,8114
5,20,7232
6,16,7204
7,3,7170
8,17,6997
9,1,6834


In [15]:
query = ''' 
    SELECT
        "MSISDN/Number" AS User,
        COUNT(*) AS SessionCount,
        SUM("Dur. (ms)") AS TotalSessionDuration
    FROM df
    GROUP BY "MSISDN/Number"
    ORDER BY TotalSessionDuration DESC;

'''

result_df = pysqldf(query)
result_df

,User,SessionCount,TotalSessionDuration
0,3.362632e+10,1083,81238323.0
1,3.362578e+10,17,18553754.0
2,3.361489e+10,17,9966898.0
3,3.376054e+10,15,9279434.0
4,3.366716e+10,13,8744914.0
...,...,...,...
106851,3.365043e+10,1,7267.0
106852,3.369839e+10,1,7258.0
106853,3.365933e+10,1,7189.0
106854,3.366566e+10,1,7146.0


### The total download and updload data per user

In [16]:
result_df = util.get_total_download_for_each_app(df, "Total DL (Bytes)", "Total UL (Bytes)")

### number of uses who have a total upload + download >= 50 MB

In [17]:
query = '''SELECT COUNT(*) AS num_users
    FROM result_df
    WHERE Total >= 100000000;
    '''

pysqldf(query)


,num_users
0,104448


### Total social media download  and upload per user

In [18]:
util.get_total_download_for_each_app(df, "Social Media DL (Bytes)", "Social Media UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,1.978664e+09,36700315.0,2.015364e+09
1,3.376054e+10,3.929765e+07,485543.0,3.978319e+07
2,3.365973e+10,3.490044e+07,511914.0,3.541236e+07
3,3.361489e+10,2.774974e+07,544800.0,2.829454e+07
4,3.362578e+10,2.656412e+07,571379.0,2.713550e+07
...,...,...,...,...
106851,3.376220e+10,1.969000e+03,5534.0,7.503000e+03
106852,3.376049e+10,2.890000e+02,7093.0,7.382000e+03
106853,3.366825e+10,2.510000e+02,4218.0,4.469000e+03
106854,3.365984e+10,1.470000e+02,1856.0,2.003000e+03


### Total youtube download and upload per user

In [19]:
util.get_total_download_for_each_app(df, "YouTube DL (Bytes)", "YouTube UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,1.264705e+10,1.204552e+10,2.469257e+10
1,3.362578e+10,2.378830e+08,2.150757e+08,4.529588e+08
2,3.376054e+10,1.940683e+08,2.022209e+08,3.962892e+08
3,3.361489e+10,1.998234e+08,1.945468e+08,3.943702e+08
4,3.367588e+10,1.586652e+08,1.587453e+08,3.174106e+08
...,...,...,...,...
106851,3.369970e+10,1.508400e+04,2.242730e+05,2.393570e+05
106852,3.376848e+10,3.387200e+04,1.926900e+05,2.265620e+05
106853,3.364809e+10,5.730500e+04,1.506500e+05,2.079550e+05
106854,3.366949e+10,3.700800e+04,1.204380e+05,1.574460e+05


### Total netflix download and upload

In [20]:
util.get_total_download_for_each_app(df, "Netflix DL (Bytes)", "Netflix UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,1.247683e+10,1.208284e+10,2.455967e+10
1,3.365973e+10,1.962772e+08,2.032419e+08,3.995191e+08
2,3.361489e+10,1.795860e+08,1.818151e+08,3.614010e+08
3,3.362578e+10,1.939122e+08,1.630684e+08,3.569806e+08
4,3.376054e+10,1.465008e+08,1.881424e+08,3.346433e+08
...,...,...,...,...
106851,3.369849e+10,2.252810e+05,5.885200e+04,2.841330e+05
106852,3.369853e+10,1.298990e+05,1.303530e+05,2.602520e+05
106853,3.366749e+10,5.416700e+04,1.944610e+05,2.486280e+05
106854,3.361588e+10,8.709300e+04,1.144710e+05,2.015640e+05


### Total Google download and upload

In [21]:
util.get_total_download_for_each_app(df, "Google DL (Bytes)", "Google UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,6.118298e+09,2.274663e+09,8.392961e+09
1,3.362578e+10,1.079047e+08,3.440318e+07,1.423079e+08
2,3.361489e+10,9.394171e+07,3.403208e+07,1.279738e+08
3,3.376054e+10,8.862653e+07,3.459657e+07,1.232231e+08
4,3.365973e+10,9.095855e+07,2.555780e+07,1.165163e+08
...,...,...,...,...
106851,3.376132e+10,6.400300e+04,4.867100e+04,1.126740e+05
106852,3.366953e+10,7.992100e+04,3.266900e+04,1.125900e+05
106853,3.367409e+10,9.632800e+04,3.498000e+03,9.982600e+04
106854,3.369800e+10,7.968800e+04,1.016800e+04,8.985600e+04


### Total Email download and upload per user

In [22]:
util.get_total_download_for_each_app(df, "Email DL (Bytes)", "Email UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,1.935661e+09,504379004.0,2.440040e+09
1,3.361489e+10,3.360721e+07,7181423.0,4.078863e+07
2,3.362578e+10,3.189541e+07,8738554.0,4.063397e+07
3,3.378632e+10,3.087691e+07,5433215.0,3.631012e+07
4,3.365973e+10,2.990765e+07,6092145.0,3.599979e+07
...,...,...,...,...
106851,3.366490e+10,6.514000e+03,22937.0,2.945100e+04
106852,3.367590e+10,1.668800e+04,10815.0,2.750300e+04
106853,3.364479e+10,2.503800e+04,1078.0,2.611600e+04
106854,3.365204e+10,1.358600e+04,12501.0,2.608700e+04


### Total gaming download and upload per user

In [23]:
util.get_total_download_for_each_app(df, "Gaming DL (Bytes)", "Gaming UL (Bytes)")

,User_MSISDN,Total_Download,Total_Upload,Total
0,3.362632e+10,4.590687e+11,8.866617e+09,4.679353e+11
1,3.361489e+10,7.622035e+09,1.273968e+08,7.749432e+09
2,3.376054e+10,7.316364e+09,1.446811e+08,7.461045e+09
3,3.362578e+10,7.171884e+09,1.547897e+08,7.326673e+09
4,3.367588e+10,6.863307e+09,1.072606e+08,6.970568e+09
...,...,...,...,...
106851,3.369806e+10,1.250150e+05,1.055888e+06,1.180903e+06
106852,3.369808e+10,8.380300e+04,8.553130e+05,9.391160e+05
106853,3.376960e+10,8.738400e+04,6.026540e+05,6.900380e+05
106854,3.368231e+10,1.322600e+05,4.631150e+05,5.953750e+05


### Other download and upload per user

In [24]:
util.get_total_download_for_each_app(df, "Other DL", "Other UL")

,User_MSISDN,Total_Download,Total_Upload,Total
0,8.823971e+14,0.0,0.0,0.0
1,3.370000e+14,0.0,0.0,0.0
2,3.197021e+12,0.0,0.0,0.0
3,3.379000e+10,0.0,0.0,0.0
4,3.379000e+10,0.0,0.0,0.0
...,...,...,...,...
106851,3.360101e+10,0.0,0.0,0.0
106852,3.360101e+10,0.0,0.0,0.0
106853,3.360100e+10,0.0,0.0,0.0
106854,3.360100e+10,0.0,0.0,0.0
